## Random Forest Classifier model to predict if rating will be >= 4

In [1]:
# Import dependencies
import pandas as pd
import pymssql
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# Connect to the pymssql database server
con = pymssql.connect(server='rubinasqlserver.database.windows.net',
                      user='group4admin',
                      password='group4@pwd',
                      database='rubinasql',
                      timeout=0,
                      login_timeout=60,
                      charset='UTF-8',
                      as_dict=True,
                      appname=None,
                      port='1433')

cursor = con.cursor()

query = "SELECT * FROM dbo.Philadelphia_data_cleaned"

cursor.execute(query)

data = cursor.fetchall()

for item in data:
    print(item)
    
con.close()

{'business_id': 'ytynqOUb3hjKeJfRj5Tshw', 'categories': 'Candy Stores, Shopping, Department Stores, Fast Food, Beer, Wine & Spirits, Fruits & Veggies, Chinese, Food, Ice Cream & Frozen Yogurt, Desserts, Seafood, Health Markets, Bagels, Cheese Shops, Shopping Centers, Chocolatiers & Shops, Meat Shops, Public Markets, Food Court, Wineries, Local Flavor, Ethnic Food, Restaurants, Specialty Food, Arts & Entertainment, Juice Bars & Smoothies, Seafood Markets, Farmers Market, Coffee & Tea, Bakeries, Food Stands, Dinner Theater, Sporting Goods, Grocery, Fashion', 'city': 'Philadelphia', 'latitude': 39.95334243774414, 'longitude': -75.15885162353516, 'name': 'Reading Terminal Market', 'review_count': 5721, 'stars_business': 4.5, 'review_id': 'kKC5pBPkUCWo6mKYFUewRw', 'stars_review': 5.0, 'text': None, 'user_id': 'mGnZFbk2gqLLtGW-mYo__A'}
{'business_id': 'ytynqOUb3hjKeJfRj5Tshw', 'categories': 'Candy Stores, Shopping, Department Stores, Fast Food, Beer, Wine & Spirits, Fruits & Veggies, Chinese

{'business_id': 'Zqd_DHAtrdzRJ8u3zwrvEA', 'categories': 'Restaurants, Delis, Sandwiches, Salad', 'city': 'Philadelphia', 'latitude': 39.953285217285156, 'longitude': -75.1594009399414, 'name': "Hershel's East Side Deli", 'review_count': 718, 'stars_business': 4.5, 'review_id': 'F_E3LTihyjF_8L755b5Kzw', 'stars_review': 5.0, 'text': None, 'user_id': 'tuRPKVjnSW3Oq5a7FpcY_g'}
{'business_id': 'Zqd_DHAtrdzRJ8u3zwrvEA', 'categories': 'Restaurants, Delis, Sandwiches, Salad', 'city': 'Philadelphia', 'latitude': 39.953285217285156, 'longitude': -75.1594009399414, 'name': "Hershel's East Side Deli", 'review_count': 718, 'stars_business': 4.5, 'review_id': '4VV07ljJ5523q3Lica8qJA', 'stars_review': 4.0, 'text': "Our first day in Philadelphia we were walking around and got pretty hungry.  I was in the mood for a sandwich and we looked up the best deli spots in PA and this place came up.  It was a bit of a walk and we ended up at a super crowded public market.   The line was kind of long for this sp

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
# Load the data into a dataframe
phl_df = pd.DataFrame(data)
phl_df.head()

,business_id,categories,city,latitude,longitude,name,review_count,stars_business,review_id,stars_review,text,user_id
0,ytynqOUb3hjKeJfRj5Tshw,"Candy Stores, Shopping, Department Stores, Fas...",Philadelphia,39.953342,-75.158852,Reading Terminal Market,5721,4.5,kKC5pBPkUCWo6mKYFUewRw,5.0,None,mGnZFbk2gqLLtGW-mYo__A
1,ytynqOUb3hjKeJfRj5Tshw,"Candy Stores, Shopping, Department Stores, Fas...",Philadelphia,39.953342,-75.158852,Reading Terminal Market,5721,4.5,sFQrhTbTah0o2kU_Pi2D0Q,5.0,One of my favorite places to go to in Philly; ...,Tu4ATXLhy8kRTjpQCnl2pA
2,ytynqOUb3hjKeJfRj5Tshw,"Candy Stores, Shopping, Department Stores, Fas...",Philadelphia,39.953342,-75.158852,Reading Terminal Market,5721,4.5,kqn1uP3LRVjVDUD44ZSu1A,5.0,This might be a bit unfair to have a single re...,vRNb2IaGlsZRA_wUf3Ov8w
3,ytynqOUb3hjKeJfRj5Tshw,"Candy Stores, Shopping, Department Stores, Fas...",Philadelphia,39.953342,-75.158852,Reading Terminal Market,5721,4.5,qMsTe9QznpNQk1AKbYLp-w,4.0,"Alright, I remember the first time I went to t...",29K-usmZfVDeIaQ85EG54A
4,ytynqOUb3hjKeJfRj5Tshw,"Candy Stores, Shopping, Department Stores, Fas...",Philadelphia,39.953342,-75.158852,Reading Terminal Market,5721,4.5,nmMIRBNONIICe7CFHnfadQ,4.0,It's an experience; to say the least! Not as ...,1jE--VcTddwXGampD23JCg


## Preprocess data

In [4]:
# View a list of all of the columns
phl_df.columns

Index(['business_id', 'categories', 'city', 'latitude', 'longitude', 'name',
       'review_count', 'stars_business', 'review_id', 'stars_review', 'text',
       'user_id'],
      dtype='object')

In [5]:
# Drop columns that are not relevant to the analysis / will confuse the model
phl_df = phl_df.drop(columns=[
    'business_id',
    'city',
    'review_id',
    'stars_review',
    'text',
    'user_id'
])
phl_df.head()

,categories,latitude,longitude,name,review_count,stars_business
0,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5
1,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5
2,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5
3,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5
4,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5


In [6]:
# Copy the stars_business column into new 'stars_binary' column
phl_df['stars_binary'] = phl_df['stars_business']
phl_df.head()

,categories,latitude,longitude,name,review_count,stars_business,stars_binary
0,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,4.5
1,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,4.5
2,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,4.5
3,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,4.5
4,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,4.5


In [7]:
# Convert 'stars_binary' column to binary values
phl_df.loc[phl_df["stars_business"] < 4, "stars_binary"] = 0
phl_df.loc[phl_df["stars_business"] >= 4, "stars_binary"] = 1
phl_df

,categories,latitude,longitude,name,review_count,stars_business,stars_binary
0,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1.0
1,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1.0
2,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1.0
3,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1.0
4,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1.0
...,...,...,...,...,...,...,...
9995,"Sandwiches, Restaurants, Cheesesteaks",39.876045,-75.243156,Geno's,174,2.0,0.0
9996,"Sandwiches, Restaurants, Cheesesteaks",39.876045,-75.243156,Geno's,174,2.0,0.0
9997,"Sandwiches, Restaurants, Cheesesteaks",39.876045,-75.243156,Geno's,174,2.0,0.0
9998,"Sandwiches, Restaurants, Cheesesteaks",39.876045,-75.243156,Geno's,174,2.0,0.0


In [8]:
# Check column data types
phl_df.dtypes

categories         object
latitude          float64
longitude         float64
name               object
review_count        int64
stars_business    float64
stars_binary      float64
dtype: object

In [9]:
# Convert 'stars_binary' column from float64 to int to get rid of the decimal
phl_df = phl_df.astype({'stars_binary' : int})
phl_df.head()

,categories,latitude,longitude,name,review_count,stars_business,stars_binary
0,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1
1,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1
2,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1
3,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1
4,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,4.5,1


In [10]:
# Drop the 'stars_business' column since the 'stars_binary' column is what we will use for the analysis
phl_df = phl_df.drop(columns=['stars_business'])
phl_df.head()

,categories,latitude,longitude,name,review_count,stars_binary
0,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,1
1,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,1
2,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,1
3,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,1
4,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,1


In [11]:
# Drop duplicate rows
phl_df = phl_df.drop_duplicates(['name'])
phl_df

,categories,latitude,longitude,name,review_count,stars_binary
0,"Candy Stores, Shopping, Department Stores, Fas...",39.953342,-75.158852,Reading Terminal Market,5721,1
7,"Sandwiches, Delis, Restaurants, Cheesesteaks",40.029495,-75.205971,Dalessandro’s Steaks & Hoagies,2686,1
11,"Italian, American (Traditional), Sandwiches, F...",39.933201,-75.159264,Pat's King of Steaks,4250,0
22,"Sandwiches, Cheesesteaks, Steakhouses, Restaur...",39.933838,-75.158813,Geno's Steaks,3401,0
27,"Food, Donuts, Food Stands, Restaurants, Specia...",39.953766,-75.159218,Beiler's Bakery,1368,1
...,...,...,...,...,...,...
9948,"Bars, Caribbean, Restaurants, Nightlife",39.950794,-75.173988,Rum Bar,174,0
9951,"Sandwiches, Food, Desserts, Coffee & Tea, Rest...",39.955349,-75.154060,Prince Tea House,176,1
9961,"Sandwiches, Burgers, Delis, Restaurants, Local...",39.948441,-75.217117,Lee's Deli,175,1
9971,"Restaurants, Desserts, Food, Italian, Pizza",39.951351,-75.179398,Sotto,175,1


In [12]:
# Value_counts the 'categories' column for preprocessing
cat_counts = phl_df.categories.value_counts()
print(cat_counts)

Restaurants, Italian                                                                                                                                                        10
Italian, Restaurants                                                                                                                                                         7
American (New), Restaurants                                                                                                                                                  7
Mexican, Restaurants                                                                                                                                                         7
Thai, Restaurants                                                                                                                                                            5
                                                                                                                             

### Define variables and Split the data into train and test sets

In [13]:
# Create our features
X = phl_df.copy()
X = X.drop(columns="stars_binary", axis=1)
X = pd.get_dummies(X)
X.head()

,latitude,longitude,review_count,"categories_Active Life, Beer Gardens, Grocery, Middle Eastern, Specialty Food, American (New), Parks, Cafes, Food, Nightlife, Restaurants, Coffee & Tea, Lebanese","categories_Active Life, Nightlife, Bars, American (Traditional), Bowling, Restaurants, American (New), Gastropubs","categories_African, Restaurants","categories_American (New), Barbeque, Restaurants","categories_American (New), Bars, Cocktail Bars, Sandwiches, Breakfast & Brunch, Nightlife, Gastropubs, Restaurants","categories_American (New), Bars, Food, Pubs, Breweries, Pizza, Restaurants, Nightlife","categories_American (New), Bars, Nightlife, Cocktail Bars, Arts & Entertainment, Lounges, Event Planning & Services, Venues & Event Spaces, Restaurants, Music Venues",...,name_Zama,name_Zavino,name_Zento Contemporary Japanese Cuisine,name_Zesto Pizza & Grill,name_Zio's Brick Oven Pizzeria,name_Zorba's Taverna,name_a.kitchen,name_bao•logy,name_honeygrow,name_sweetgreen
0,39.953342,-75.158852,5721,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,40.029495,-75.205971,2686,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,39.933201,-75.159264,4250,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,39.933838,-75.158813,3401,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,39.953766,-75.159218,1368,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Create our target
y = phl_df["stars_binary"].ravel()
y[:5]

array([1, 1, 0, 0, 1])

In [15]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Check the shape of X_train
X_train.shape

(720, 1851)

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Create, Train, and Evaluate the model

In [17]:
# Define the logistic regression model
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [18]:
# Train the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [19]:
# Evaluate the model
predictions = rf_model.predict(X_test_scaled)

In [20]:
# Calculate the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,2,83
Actual 1,1,155


In [21]:
# Calculate the Random Forest Classifer model accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [22]:
# Display the results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2,83
Actual 1,1,155


Accuracy Score : 0.6514522821576764
Classification Report
              precision    recall  f1-score   support

           0       0.67      0.02      0.05        85
           1       0.65      0.99      0.79       156

    accuracy                           0.65       241
   macro avg       0.66      0.51      0.42       241
weighted avg       0.66      0.65      0.53       241



In [23]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.05917271, 0.06466123, 0.06201912, ..., 0.00026765, 0.        ,
       0.00014648])